In [36]:
import pyarrow.parquet as pq

# Path to the Parquet file
file_path = "/Users/Macington/Documents/Projects/Project Gambit/Games/Ply.parquet"

# Read the Parquet file into an Arrow table
table = pq.read_table(file_path)

# If you want to convert the Arrow table to a Pandas DataFrame
df = table.to_pandas()
print(df)

              game_id    ply             board_sum  centipawn_evaluation  \
0        1.844646e+19    NaN  18446462598732906495                   NaN   
2705610  3.689293e+19    NaN  18446462598732906495                   NaN   
2705611  3.689293e+19    1.0  18446462598735003583                  18.0   
2705612  3.689293e+19    NaN  18446462598732906495                   NaN   
2705613  3.689293e+19    1.0  18446462598749683455                 -40.0   
...               ...    ...                   ...                   ...   
2798774  3.317226e+21  236.0  13835102769989829121                -433.0   
2798775  3.317226e+21  237.0  13835102769985634817                -590.0   
2798776  3.317226e+21  238.0  13835102767846539777                -521.0   
2798777  3.317226e+21  239.0  13835102767578105345                  -2.0   
2798778  3.317226e+21  240.0  13835102492700198401                  -1.0   

         board_sum_in_context  
0                1.844646e+19  
2705610          1.8446

In [34]:
import pyarrow.parquet as pq
import pandas as pd

# Path to the original Parquet file
file_path = "/Users/Macington/Documents/Projects/Project Gambit/Games/Database.parquet"

# Read the original Parquet file
table = pq.read_table(file_path)

# Separate the columns
table1_columns = ['game_id', 'pgn']
table2_columns = ['game_id'] + [col for col in table.schema.names if col not in table1_columns + ['total_ply']]

# Create Pandas DataFrames for sorting and deduplication
df1 = table.select(table1_columns).to_pandas()
df2 = table.select(table2_columns).to_pandas()

# Sort and deduplicate table1 by 'game_id'
df1 = df1.drop_duplicates(subset='game_id').sort_values(by='game_id')

# Sort table2 by 'game_id' and then 'ply' with nulls first
df2 = df2.sort_values(by=['game_id', 'ply'], ascending=[True, True], na_position='first')

# Convert back to pyarrow Tables
table1 = pa.Table.from_pandas(df1)
table2 = pa.Table.from_pandas(df2)

# Output paths
output_directory = "/Users/Macington/Documents/Projects/Project Gambit/Games/"
output_file1 = f"{output_directory}PGNs.parquet"
output_file2 = f"{output_directory}Ply.parquet"

# Write the separated tables to Parquet files with snappy compression
pq.write_table(table1, output_file1, compression='brotli')
pq.write_table(table2, output_file2, compression='brotli')
